# **Mount & imports**

In [1]:
# Import images from google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
#imports:
import os
import cv2
import numpy as np
import sys
from IPython.display import Image
import yaml
import shutil
from sklearn.model_selection import train_test_split
import random
from collections import defaultdict
from random import shuffle

In [3]:
!nvidia-smi

Thu Mar 27 09:37:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


# **YOLOV8**

INSTALL

In [5]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/235.7 GB disk)


In [6]:
from ultralytics import YOLO

from IPython.display import display, Image

In [7]:
!pip install roboflow

from roboflow import Roboflow
import roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


roboflow - 2023 data

In [1]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet


roboflow.login()

rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")

project = rf.workspace("citrus-a0lk2").project("citrus_prediction_data")
dataset = project.version(8).download("yolov8-obb")

import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)


mkdir: cannot create directory ‘{HOME}/datasets’: No such file or directory
[Errno 2] No such file or directory: '{HOME}/datasets'
/content


NameError: name 'roboflow' is not defined

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("citrus_prediction_data")
version = project.version(8)
dataset = version.download("yolov8-obb")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Citrus_prediction_data-8 in yolov8-obb:: 100%|██████████| 2350/2350 [00:00<00:00, 8029.29it/s]


augmented 2023

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("citrus_prediction_data")
version = project.version(9)
dataset = version.download("yolov8-obb")

loading Roboflow workspace...
loading Roboflow project...


roboflow - 2024 data

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("citrus_prediciton_data_2024")
version = project.version(4)
dataset = version.download("yolov8-obb")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to citrus_prediciton_data_2024-4 in yolov8-obb:: 100%|██████████| 2492/2492 [00:00<00:00, 2502.59it/s]


augmented 2024 data

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("citrus_prediciton_data_2024")
version = project.version(3)
dataset = version.download("yolov8-obb")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to citrus_prediciton_data_2024-3 in yolov8-obb:: 100%|██████████| 5954/5954 [00:01<00:00, 4016.33it/s]


In [3]:
import os
import shutil
from pathlib import Path
import yaml

# Define the original dataset paths
dataset_paths = [
    "/content/Citrus_prediction_data-9",
    "/content/citrus_prediciton_data_2024-3"
]

# Define the destination combined dataset path
combined_dataset_path = "/content/citrus_2023_2024"
os.makedirs(combined_dataset_path, exist_ok=True)

# Define subsets to combine
subsets = ['train', 'valid', 'test']

# Create folders in the new combined dataset
for subset in subsets:
    for subfolder in ['images', 'labels']:
        os.makedirs(os.path.join(combined_dataset_path, subset, subfolder), exist_ok=True)

# Function to copy data
def copy_data(src_path, subset, combined_path):
    for subfolder in ['images', 'labels']:
        src_folder = os.path.join(src_path, subset, subfolder)
        dst_folder = os.path.join(combined_path, subset, subfolder)
        for file in os.listdir(src_folder):
            src_file = os.path.join(src_folder, file)
            dst_file = os.path.join(dst_folder, file)
            # Prevent overwriting by renaming if needed
            if os.path.exists(dst_file):
                filename, ext = os.path.splitext(file)
                counter = 1
                while os.path.exists(dst_file):
                    dst_file = os.path.join(dst_folder, f"{filename}_{counter}{ext}")
                    counter += 1
            shutil.copy2(src_file, dst_file)

# Copy the data
for dataset_path in dataset_paths:
    for subset in subsets:
        copy_data(dataset_path, subset, combined_dataset_path)

# Load and modify the YAML file from the first dataset
yaml_path = os.path.join(dataset_paths[0], 'data.yaml')
with open(yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# Update YAML paths to new combined dataset structure
data_yaml['train'] = os.path.join(combined_dataset_path, 'train', 'images')
data_yaml['val'] = os.path.join(combined_dataset_path, 'valid', 'images')
data_yaml['test'] = os.path.join(combined_dataset_path, 'test', 'images')

# Save the new YAML file
new_yaml_path = os.path.join(combined_dataset_path, 'data.yaml')
with open(new_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

# Show the new structure
combined_structure = {}
for subset in subsets:
    combined_structure[subset] = {
        'images': len(os.listdir(os.path.join(combined_dataset_path, subset, 'images'))),
        'labels': len(os.listdir(os.path.join(combined_dataset_path, subset, 'labels')))
    }

import pandas as pd

df = pd.DataFrame(combined_structure).T
print("✅ Combined Dataset Structure:")
display(df)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Citrus_prediction_data-9/train/images'

calculate how many images in each set

In [13]:
import os

# Define the paths to the image directories
train_images_path = '/content/citrus_prediciton_data_2024-4/train/images'
test_images_path = '/content/citrus_prediciton_data_2024-4/test/images'
valid_images_path = '/content/citrus_prediciton_data_2024-4/valid/images'

# Function to count the number of image files in a directory
def count_images(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])

# Calculate the number of images in each folder
train_images_count = count_images(train_images_path)
test_images_count = count_images(test_images_path)
valid_images_count = count_images(valid_images_path)

print(f"Number of images in train folder: {train_images_count}")
print(f"Number of images in test folder: {test_images_count}")
print(f"Number of images in valid folder: {valid_images_count}")



# Define the paths to the image directories
train_images_path = '/content/Citrus_prediction_data-8/train/images'
test_images_path = '/content/Citrus_prediction_data-8/test/images'
valid_images_path = '/content/Citrus_prediction_data-8/valid/images'

# Function to count the number of image files in a directory
def count_images(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])

# Calculate the number of images in each folder
train_images_count = count_images(train_images_path)
test_images_count = count_images(test_images_path)
valid_images_count = count_images(valid_images_path)

print(f"Number of images in train folder: {train_images_count}")
print(f"Number of images in test folder: {test_images_count}")
print(f"Number of images in valid folder: {valid_images_count}")


Number of images in train folder: 869
Number of images in test folder: 124
Number of images in valid folder: 247
Number of images in train folder: 817
Number of images in test folder: 119
Number of images in valid folder: 233


# **Functions**

In [10]:
### calculate how many objects in each class
def count_objects(label_folder, class_counts):
    """ Count objects in each class from label files in the given directory. """
    label_files = [file for file in os.listdir(label_folder) if file.endswith('.txt')]
    for label_file in label_files:
        label_file_path = os.path.join(label_folder, label_file)
        with open(label_file_path, 'r') as f:
            for line in f:
                class_label = line.strip().split()[0]
                if class_label in class_counts:
                    class_counts[class_label] += 1
                else:
                    class_counts[class_label] = 1
    return class_counts


def print_class_counts(train_label_folder, valid_label_folder, test_label_folder):
    """ Process label folders and print class counts for each dataset. """
    folders = [
        (train_label_folder, "train"),
        (valid_label_folder, "valid"),
        (test_label_folder, "test")
    ]

    total_class_counts = {}

    for folder, folder_name in folders:
        if os.path.exists(folder):
            class_counts = count_objects(folder, {})
            print(f"\nCounts for {folder_name} set:")
            for class_label, count in class_counts.items():
                print(f"Total objects in class '{class_label}': {count}")
                # Update total counts across datasets
                total_class_counts[class_label] = total_class_counts.get(class_label, 0) + count
        else:
            print(f"The directory {folder} does not exist. Please check the path and try again.")

    # Output total counts across all datasets
    print("\nTotal counts across all datasets:")
    for class_label, count in total_class_counts.items():
        print(f"Total objects in class '{class_label}': {count}")


### duplicate directory
def duplicate_directory(source_path, target_path):
    """
    Copies the entire directory from source_path to target_path.
    If the target directory already exists, it is deleted and recreated.
    """
    # Check if the target path already exists.
    if os.path.exists(target_path):
        print(f"Target directory {target_path} already exists. Deleting and recreating.")
        shutil.rmtree(target_path)  # Remove the existing directory
        shutil.copytree(source_path, target_path)  # Copy again
        print(f"Directory re-copied from {source_path} to {target_path}.")
    else:
        # Copy the directory if it does not exist
        shutil.copytree(source_path, target_path)
        print(f"Directory copied from {source_path} to {target_path}.")


### update "path" in Yaml files for each dataset
def update_yaml_path(yaml_file_path):
    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Update the 'path' field with the directory of the YAML file
    data['path'] = os.path.dirname(yaml_file_path)

    # Write the updated data back to the YAML file
    with open(yaml_file_path, 'w') as file:
        yaml.safe_dump(data, file, default_flow_style=False, sort_keys=False)



### function that balances between classes as the amount as minor class (reduce labels)
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
from random import shuffle
from collections import defaultdict
def balance_and_split_dataset(base_path, ratio="60:20:20"):
    train_ratio, valid_ratio, test_ratio = map(int, ratio.split(':'))
    total_ratio = train_ratio + valid_ratio + test_ratio

    # Calculate the actual ratios from the provided percentages
    train_ratio /= total_ratio
    valid_ratio /= total_ratio
    test_ratio /= total_ratio

    # Gather all label files from the dataset
    class_files = defaultdict(list)
    for subset in ['train', 'valid', 'test']:
        dir_path = os.path.join(base_path, subset, 'labels')
        for filename in os.listdir(dir_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(dir_path, filename)
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    if not lines:
                        os.remove(file_path)  # Remove empty files
                        continue
                    # Assume the class label is the first number on the first line
                    class_label = lines[0].strip().split()[0]
                    class_files[class_label].append(file_path)

    # Determine the minimum number of files in any class across the dataset
    min_count = min(len(files) for files in class_files.values())

    # Reduce all classes to this minimum count
    for label, files in class_files.items():
        if len(files) > min_count:
            shuffle(files)  # Shuffle before removing
            excess_count = len(files) - min_count
            files_to_remove = files[-excess_count:]  # Get excess files to remove
            for file in files_to_remove:
                os.remove(file)  # Remove excess files
            class_files[label] = files[:-excess_count]  # Keep the remaining files

    # Distribute the balanced files into train, valid, and test directories
    for label, files in class_files.items():
        shuffle(files)  # Shuffle to randomize distribution
        train_end = int(len(files) * train_ratio)
        valid_end = train_end + int(len(files) * valid_ratio)

        # Define the destination folders for train, valid, and test
        train_files = files[:train_end]
        valid_files = files[train_end:valid_end]
        test_files = files[valid_end:]

        # Function to move files to new destination
        def move_files(files, target_folder):
            for file in files:
                target_path = os.path.join(base_path, target_folder, 'labels', os.path.basename(file))
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                shutil.move(file, target_path)

        # Move files to their respective new folders
        move_files(train_files, 'train')
        move_files(valid_files, 'valid')
        move_files(test_files, 'test')

In [15]:
train_label_folder = '/content/citrus_2023_2024/train/labels'
valid_label_folder = '/content/citrus_2023_2024/valid/labels'
test_label_folder = '/content/citrus_2023_2024/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 1878
Total objects in class '0': 112

Counts for valid set:
Total objects in class '1': 515
Total objects in class '0': 19

Counts for test set:
Total objects in class '1': 257
Total objects in class '0': 10

Total counts across all datasets:
Total objects in class '1': 2650
Total objects in class '0': 141


In [16]:
train_label_folder = '/content/citrus_prediciton_data_2024-4/train/labels'
valid_label_folder = '/content/citrus_prediciton_data_2024-4/valid/labels'
test_label_folder = '/content/citrus_prediciton_data_2024-4/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 1037
Total objects in class '0': 80

Counts for valid set:
Total objects in class '1': 283
Total objects in class '0': 8

Counts for test set:
Total objects in class '1': 142
Total objects in class '0': 2

Total counts across all datasets:
Total objects in class '1': 1462
Total objects in class '0': 90


In [ ]:
train_label_folder = '/content/Citrus_prediction_data-8/train/labels'
valid_label_folder = '/content/Citrus_prediction_data-8/valid/labels'
test_label_folder = '/content/Citrus_prediction_data-8/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 841
Total objects in class '0': 32

Counts for valid set:
Total objects in class '1': 232
Total objects in class '0': 11

Counts for test set:
Total objects in class '1': 115
Total objects in class '0': 8

Total counts across all datasets:
Total objects in class '1': 1188
Total objects in class '0': 51


delete if exist and create dataset_augment and dataset_balanced folders

In [ ]:
# Define the source directory and the target directories
source_directory = '/content/Citrus_prediction_data-9'
target_directory_augment = '/content/dataset_augment'
target_directory_balanced = '/content/dataset_balanced'

# Duplicate the source directory to dataset-augment
duplicate_directory(source_directory, target_directory_augment)

# Duplicate the source directory to dataset-balanced
duplicate_directory(source_directory, target_directory_balanced)


Directory copied from /content/Citrus_prediction_data-9 to /content/dataset_augment.
Directory copied from /content/Citrus_prediction_data-9 to /content/dataset_balanced.


Update Yaml's "path" for each dataset

In [ ]:
yaml_file_path = '/content/dataset_augment/data.yaml'
update_yaml_path(yaml_file_path)


yaml_file_path = '/content/dataset_balanced/data.yaml'
update_yaml_path(yaml_file_path)

In [ ]:
train_label_folder = '/content/dataset_balanced/train/labels'
valid_label_folder = '/content/dataset_balanced/valid/labels'
test_label_folder = '/content/dataset_balanced/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 2517
Total objects in class '0': 96

Counts for valid set:
Total objects in class '1': 232
Total objects in class '0': 11

Counts for test set:
Total objects in class '1': 115
Total objects in class '0': 8

Total counts across all datasets:
Total objects in class '1': 2864
Total objects in class '0': 115


balance classes  by minor class, train 60 , valid 20 , test 20

In [ ]:
dataset_path = '/content/dataset_balanced'
balance_and_split_dataset(dataset_path, "70:20:10")

In [ ]:
train_label_folder = '/content/dataset_balanced/train/labels'
valid_label_folder = '/content/dataset_balanced/valid/labels'
test_label_folder = '/content/dataset_balanced/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 84
Total objects in class '0': 80

Counts for valid set:
Total objects in class '0': 23
Total objects in class '1': 25

Counts for test set:
Total objects in class '0': 12
Total objects in class '1': 15

Total counts across all datasets:
Total objects in class '1': 124
Total objects in class '0': 115


# **Augment Data to YOLO**

In [ ]:
train_label_folder = '/content/dataset_augment/train/labels'
valid_label_folder = '/content/dataset_augment/valid/labels'
test_label_folder = '/content/dataset_augment/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 2517
Total objects in class '0': 96

Counts for valid set:
Total objects in class '1': 232
Total objects in class '0': 11

Counts for test set:
Total objects in class '1': 115
Total objects in class '0': 8

Total counts across all datasets:
Total objects in class '1': 2864
Total objects in class '0': 115


augment images

In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms

# Define the paths
train_images_path = '/content/dataset_augment/train/images'
train_labels_path = '/content/dataset_augment/train/labels'

# Define augmentation transformations
augment_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.Lambda(lambda img: img.rotate(90, expand=True)),  # 90° clockwise
        transforms.Lambda(lambda img: img.rotate(-90, expand=True))  # 90° counter-clockwise
    ], p=0.5),  # Randomly apply 90° rotations with 50% chance
    transforms.RandomRotation(degrees=15),  # Random rotation between -15° and +15°
])

# Number of augmented copies to create for each class "0" image
num_augmentations = 5  # Set to however many you need

# Function to augment and save images of class "0"
def augment_class_zero_images(images_path, labels_path, num_augmentations):
    label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
    for label_file in label_files:
        label_path = os.path.join(labels_path, label_file)
        with open(label_path, 'r') as file:
            lines = file.readlines()
            if any(line.split()[0] == '0' for line in lines):  # Check if any object is of class "0"
                image_file = label_file.replace('.txt', '.jpg')
                image_path = os.path.join(images_path, image_file)
                if os.path.exists(image_path):
                    image = Image.open(image_path).convert('RGB')
                    for i in range(num_augmentations):
                        augmented_image = augment_transforms(image)
                        new_image_name = f'aug_{i}_{image_file}'
                        new_image_path = os.path.join(images_path, new_image_name)
                        new_label_path = os.path.join(labels_path, new_image_name.replace('.jpg', '.txt'))
                        augmented_image.save(new_image_path)
                        with open(new_label_path, 'w') as new_label_file:
                            new_label_file.writelines(lines)

# Run the function
augment_class_zero_images(train_images_path, train_labels_path, num_augmentations)


In [ ]:
train_label_folder = '/content/dataset_augment/train/labels'
valid_label_folder = '/content/dataset_augment/valid/labels'
test_label_folder = '/content/dataset_augment/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 2517
Total objects in class '0': 576

Counts for valid set:
Total objects in class '1': 232
Total objects in class '0': 11

Counts for test set:
Total objects in class '1': 115
Total objects in class '0': 8

Total counts across all datasets:
Total objects in class '1': 2864
Total objects in class '0': 595


balance data after augmentations on class "0"

In [ ]:
dataset_path = '/content/dataset_augment'
balance_and_split_dataset(dataset_path, "70:20:10")

In [ ]:
train_label_folder = '/content/dataset_augment/train/labels'
valid_label_folder = '/content/dataset_augment/valid/labels'
test_label_folder = '/content/dataset_augment/test/labels'
print_class_counts(train_label_folder, valid_label_folder, test_label_folder)


Counts for train set:
Total objects in class '1': 376
Total objects in class '0': 357

Counts for valid set:
Total objects in class '0': 119
Total objects in class '1': 126

Counts for test set:
Total objects in class '1': 124
Total objects in class '0': 119

Total counts across all datasets:
Total objects in class '1': 626
Total objects in class '0': 595


# **Train**

augmented 2023

In [ ]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/Citrus_prediction_data-9"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb9",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 92.4MB/s]

New https://pypi.org/project/ultralytics/8.3.84 available 😃 Update with 'pip install -U ultralytics'


Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/Citrus_prediction_data-9/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb9, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

100%|██████████| 755k/755k [00:00<00:00, 21.0MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

    100/100      2.17G     0.2488     0.4023      1.909         19        640: 100%|██████████| 153/153 [00:43<00:00,  3.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  5.72it/s]

                   all        233        243      0.916      0.856      0.874       0.87



100 epochs completed in 1.424 hours.
Optimizer stripped from /content/runs/obb9/train/weights/last.pt, 6.6MB
Optimizer stripped from /content/runs/obb9/train/weights/best.pt, 6.6MB

Validating /content/runs/obb9/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:06<00:00,  2.24it/s]


                   all        233        243      0.973      0.864      0.904        0.9
      Cracked_Mandarin         11         11      0.987      0.727      0.829      0.829
              Mandarin        222        232      0.959          1       0.98      0.972
Speed: 0.4ms preprocess, 3.9ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to /content/runs/obb9/train
Training completed. Results saved in /content/Citrus_prediction_data-9/runs/train_20250306_100915


train 2024 data original - no augments

In [11]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/datasets/citrus_prediciton_data_2024-4"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 92.1MB/s]


New https://pypi.org/project/ultralytics/8.3.95 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/datasets/citrus_prediciton_data_2024-4/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

100%|██████████| 755k/755k [00:00<00:00, 21.7MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 107MB/s]


...

    100/100      2.24G     0.2047     0.3866      1.796          5        640: 100%|██████████| 55/55 [00:14<00:00,  3.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  5.58it/s]

                   all        247        289      0.953      0.776      0.843      0.837



100 epochs completed in 0.542 hours.
Optimizer stripped from /content/runs/obb_2024/train/weights/last.pt, 6.5MB
Optimizer stripped from /content/runs/obb_2024/train/weights/best.pt, 6.5MB

Validating /content/runs/obb_2024/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:05<00:00,  2.91it/s]


                   all        247        289      0.959      0.775      0.843      0.837
      Cracked_Mandarin          8          8      0.922      0.625        0.7      0.696
              Mandarin        242        281      0.996      0.925      0.987      0.979
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to /content/runs/obb_2024/train
Training completed. Results saved in /content/datasets/citrus_prediciton_data_2024-4/runs/train_20250324_163159


augmentes 2024

In [8]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/citrus_prediciton_data_2024-3"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 113MB/s]


New https://pypi.org/project/ultralytics/8.3.96 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/citrus_prediciton_data_2024-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

100%|██████████| 755k/755k [00:00<00:00, 21.6MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 109MB/s]


...

    100/100      2.19G     0.2505     0.4406      1.753          9        640: 100%|██████████| 163/163 [00:53<00:00,  3.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  5.19it/s]

                   all        247        289      0.911      0.773       0.84      0.838



100 epochs completed in 1.694 hours.
Optimizer stripped from /content/runs/obb_2024/train/weights/last.pt, 6.5MB
Optimizer stripped from /content/runs/obb_2024/train/weights/best.pt, 6.5MB

Validating /content/runs/obb_2024/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:07<00:00,  2.14it/s]


                   all        247        289      0.908      0.772      0.843      0.841
      Cracked_Mandarin          8          8      0.833      0.625      0.703      0.703
              Mandarin        242        281      0.983      0.918      0.984      0.979
Speed: 0.2ms preprocess, 4.1ms inference, 0.0ms loss, 8.7ms postprocess per image
Results saved to /content/runs/obb_2024/train
Training completed. Results saved in /content/citrus_prediciton_data_2024-3/runs/train_20250325_073641


In [ ]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/citrus_prediciton_data_2024-1"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 112MB/s]

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/citrus_prediciton_data_2024-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 112MB/s]
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUB

AMP: checks passed ✅


...

  0%|          | 0/73 [00:00<?, ?it/s]Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
    100/100      2.15G     0.2008     0.3973      1.806         17        640: 100%|██████████| 73/73 [00:20<00:00,  3.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.68it/s]

                   all         48         62       0.81      0.942      0.984      0.979



100 epochs completed in 0.677 hours.
Optimizer stripped from /content/runs/obb_2024/train/weights/last.pt, 6.7MB
Optimizer stripped from /content/runs/obb_2024/train/weights/best.pt, 6.7MB

Validating /content/runs/obb_2024/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CON

                   all         48         62      0.961      0.944      0.989      0.983
      Cracked_Mandarin          2          2       0.94          1      0.995      0.995
              Mandarin         46         60      0.982      0.889      0.983      0.971
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 8.9ms postprocess per image
Results saved to /content/runs/obb_2024/train
Training completed. Results saved in /content/citrus_prediciton_data_2024-1/runs/train_20250322_090339


augmented 2024 data

In [16]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/datasets/citrus_prediciton_data_2024-3"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


New https://pypi.org/project/ultralytics/8.3.95 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/datasets/citrus_prediciton_data_2024-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

train: Scanning /content/datasets/citrus_prediciton_data_2024-3/train/labels... 2600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:03<00:00, 725.86it/s]

train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/10_e_2_20240927_RGB_JPG.rf.3aef16fc3912bbeb0282cdb99c508431.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/10_e_2_20240927_RGB_JPG.rf.8e11f1b70c67992becc75e916f4dbd81.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/10_e_2_20240927_RGB_JPG.rf.a8e27f810e9dcac7cea3b84806ba3914.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/11_w_2_20240927_RGB_JPG.rf.111265b279f2a7172869ae658c0e386b.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/11_w_2_20240927_RGB_JPG.rf.39de69800bbb4d366617295ac9eb92b5.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/citrus_prediciton_data_2024-3/train/images/11_w_2_20240927_RGB_JPG.rf.b32bd59ae16c19f2c61e3602eed422df.jpg: 1

train: New cache created: /content/datasets/citrus_prediciton_data_2024-3/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


Argument(s) 'quality_lower' are not valid for transform ImageCompression
val: Scanning /content/datasets/citrus_prediciton_data_2024-3/valid/labels... 247 images, 0 backgrounds, 0 corrupt: 100%|██████████| 247/247 [00:00<00:00, 463.88it/s]

...

    100/100      2.29G     0.2457     0.4308      1.753          9        640: 100%|██████████| 163/163 [00:46<00:00,  3.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  6.14it/s]

                   all        247        289      0.982      0.778      0.828      0.825



100 epochs completed in 1.498 hours.
Optimizer stripped from /content/runs/obb_2024/train2/weights/last.pt, 6.5MB
Optimizer stripped from /content/runs/obb_2024/train2/weights/best.pt, 6.5MB

Validating /content/runs/obb_2024/train2/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:06<00:00,  2.39it/s]


                   all        247        289      0.981      0.779      0.833       0.83
      Cracked_Mandarin          8          8      0.982      0.625      0.684      0.684
              Mandarin        242        281      0.981      0.932      0.982      0.977
Speed: 0.4ms preprocess, 3.7ms inference, 0.0ms loss, 8.1ms postprocess per image
Results saved to /content/runs/obb_2024/train2
Training completed. Results saved in /content/datasets/citrus_prediciton_data_2024-3/runs/train_20250324_171809


run with blanced data

In [ ]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/dataset_balanced"
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # e.g., "/content/Citrus_prediction_data-8/data.yaml"
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


New https://pypi.org/project/ultralytics/8.3.85 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/dataset_balanced/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:1

AMP: checks passed ✅


train: Scanning /content/dataset_balanced/train/labels... 136 images, 2311 backgrounds, 0 corrupt: 100%|██████████| 2447/2447 [00:00<00:00, 3656.91it/s]


train: New cache created: /content/dataset_balanced/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

  0%|          | 0/153 [00:00<?, ?it/s]Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
    100/100      2.37G     0.3004      3.367      1.297          3        640: 100%|██████████| 153/153 [00:44<00:00,  3.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.92it/s]

                   all        233          5      0.622       0.25      0.077      0.064



100 epochs completed in 1.434 hours.
Optimizer stripped from /content/runs/obb/train/weights/last.pt, 6.6MB
Optimizer stripped from /content/runs/obb/train/weights/best.pt, 6.6MB

Validating /content/runs/obb/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:1

                   all        233          5    0.00979          1      0.265      0.238
      Cracked_Mandarin          2          2     0.0083          1      0.502      0.451
              Mandarin          3          3     0.0113          1     0.0284     0.0248
Speed: 0.5ms preprocess, 4.3ms inference, 0.0ms loss, 10.1ms postprocess per image
Results saved to /content/runs/obb/train
Training completed. Results saved in /content/dataset_balanced/runs/train_20250308_164623


Train on combined dataset 2023+ 2024

In [ ]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/citrus_2023_2024"
data_yaml_path = os.path.join(dataset_path, "data.yaml")
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name-+0
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 316MB/s]


New https://pypi.org/project/ultralytics/8.3.91 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/citrus_2023_2024/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

100%|██████████| 755k/755k [00:00<00:00, 111MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 369MB/s]


...

    100/100      2.22G     0.1946     0.3671      1.811          1        640: 100%|██████████| 125/125 [00:36<00:00,  3.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.86it/s]

                   all        281        305      0.888      0.734      0.846      0.843



100 epochs completed in 1.190 hours.
Optimizer stripped from /content/runs/obb_2024/train/weights/last.pt, 6.6MB
Optimizer stripped from /content/runs/obb_2024/train/weights/best.pt, 6.6MB

Validating /content/runs/obb_2024/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:05<00:00,  3.40it/s]


                   all        281        305      0.849      0.816      0.886      0.882
      Cracked_Mandarin         13         13      0.726      0.692      0.787      0.787
              Mandarin        268        292      0.972      0.939      0.985      0.977
Speed: 0.4ms preprocess, 3.1ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to /content/runs/obb_2024/train
Training completed. Results saved in /content/citrus_2023_2024/runs/train_20250316_092021


In [4]:
import os
from datetime import datetime
from ultralytics import YOLO

# Paths and parameters
dataset_path = "/content/citrus_2023_2024"
data_yaml_path = os.path.join(dataset_path, "data.yaml")
model_weights = "yolov8n-obb.pt"
epochs = 100
img_size = 640

# Verify the dataset directory and data.yaml exist
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist. Skipping.")
else:
    if not os.path.isfile(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}. Skipping.")
    else:
        # Create a unique run directory based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(dataset_path, f"runs/train_{timestamp}")
        os.makedirs(run_dir, exist_ok=True)

        # Initialize the model
        model = YOLO(model_weights)

        # Train
        results = model.train(
            data=data_yaml_path,  # Use the correct variable name
            epochs=epochs,
            imgsz=img_size,
            project="/content/runs/obb_2024",  # Specify a project folder
            name="train",                 # Specify a run name
            cls=1.2
        )

        print(f"Training completed. Results saved in {run_dir}")


100%|██████████| 6.26M/6.26M [00:00<00:00, 59.1MB/s]


New https://pypi.org/project/ultralytics/8.3.96 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/citrus_2023_2024/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/obb_2024, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

100%|██████████| 755k/755k [00:00<00:00, 28.4MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 99.5MB/s]


...

    100/100      2.19G     0.2053     0.3951      1.847          7        640: 100%|██████████| 106/106 [00:30<00:00,  3.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:05<00:00,  5.50it/s]

                   all        480        532      0.934      0.736      0.801      0.797



100 epochs completed in 1.099 hours.
Optimizer stripped from /content/runs/obb_2024/train/weights/last.pt, 6.7MB
Optimizer stripped from /content/runs/obb_2024/train/weights/best.pt, 6.7MB

Validating /content/runs/obb_2024/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:09<00:00,  3.10it/s]


                   all        480        532      0.963      0.734      0.834       0.83
      Cracked_Mandarin         19         19       0.95      0.526      0.681      0.681
              Mandarin        464        513      0.977      0.942      0.986      0.979
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to /content/runs/obb_2024/train
Training completed. Results saved in /content/citrus_2023_2024/runs/train_20250326_074353


In [ ]:
from ultralytics import YOLO

# Path to best model weights
best_weights_path = "/content/runs/obb9/train/weights/best.pt"

# Path to test images directory (modify this to match your dataset)
test_images_path = "/content/Citrus_prediction_data-9/test/images"

# Load trained model
model = YOLO(best_weights_path)

# Run inference on test dataset
results = model.predict(
    source=test_images_path,  # Directory containing test images
    save=True,                # Save results
    imgsz=640,                # Image size (should match training size)
    conf=0.25                 # Confidence threshold (adjust if needed)
)

print("Inference completed. Check the 'runs/predict' directory for results.")


save test result and weights in drive

In [5]:
source_folder = "/content/runs"
destination_folder = "/content/gdrive/Othercomputers/My PC/Thesis/Yolo/YoloV8/2023+2024/original/runs"
shutil.copytree(source_folder, destination_folder)

'/content/gdrive/Othercomputers/My PC/Thesis/Yolo/YoloV8/2023+2024/original/runs'

# **RUN on balanced and augmented - this code created and didn't used eventually because after augmenting the positions of bounding boxes may changed.**

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/dataset_augment

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-obb.pt')

results = model.train(data=f"{dataset.location}/data.yaml", epochs=10, imgsz=640)

In [ ]:
model = YOLO('/content/dataset_augment/runs/obb/train/weights/best.pt')

import os
import random

random_file = random.choice(os.listdir(f"{dataset.location}/test/images"))
file_name = os.path.join(f"{dataset.location}/test/images", random_file)

results = model(file_name)

print(results[0])

augment class 0 - old

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image, ImageEnhance
import shutil

def add_gaussian_noise(image):
    row, col, ch = image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)

def augment_image(image_path, method):
    # Load the original image
    image = cv2.imread(image_path)
    pil_img = Image.open(image_path)

    if method == 'gaussian_small':
        blurred = cv2.GaussianBlur(image, (5, 5), 0)
        return Image.fromarray(blurred)
    elif method == 'gaussian_large':
        blurred = cv2.GaussianBlur(image, (11, 11), 0)
        return Image.fromarray(blurred)
    elif method == 'median':
        # Apply Median Blur
        blurred = cv2.medianBlur(image, 5)
        return Image.fromarray(blurred)
    elif method == 'noise':
        noisy = add_gaussian_noise(image)
        return Image.fromarray(noisy)
    elif method == 'brighten':
        enhancer = ImageEnhance.Brightness(pil_img)
        return enhancer.enhance(1.5)

def process_images(label_folder, image_folder):
    for label_file in os.listdir(label_folder):
        if label_file.endswith('.txt'):
            label_path = os.path.join(label_folder, label_file)
            with open(label_path, 'r') as file:
                if any('0' in line.split()[0] for line in file):
                    image_file = label_file.replace('.txt', '.jpg')
                    image_path = os.path.join(image_folder, image_file)

                    for aug_method in ['gaussian_small', 'gaussian_large', 'median', 'noise', 'brighten']:
                        augmented_image = augment_image(image_path, aug_method)
                        if augmented_image is not None:
                            new_image_name = image_file.replace('.jpg', f'_{aug_method}.jpg')
                            new_label_name = label_file.replace('.txt', f'_{aug_method}.txt')

                            # Save the augmented image
                            augmented_image.save(os.path.join(image_folder, new_image_name))
                            # Copy the original label file for the augmented image
                            shutil.copy(label_path, os.path.join(label_folder, new_label_name))

# Directory setup
label_folder = '/content/datasets/Citrus_prediction_data-8/train/labels'
image_folder = '/content/datasets/Citrus_prediction_data-8/train/images'

# Function call
process_images(label_folder, image_folder)


calculate how many object in train in each class after augmentation

In [ ]:
import os

# Function to count objects in each class
def count_objects(label_folder):
    # Initialize a dictionary to hold the count of each class
    class_counts = {'0': 0, '1': 0}

    # List all label files in the label folder
    label_files = [file for file in os.listdir(label_folder) if file.endswith('.txt')]

    # Loop through each label file and count classes
    for label_file in label_files:
        label_file_path = os.path.join(label_folder, label_file)
        with open(label_file_path, 'r') as f:
            for line in f:
                class_label = line.strip().split()[0]
                if class_label in class_counts:
                    class_counts[class_label] += 1
    return class_counts

# Correct path as confirmed by the user
train_label_folder = '/content/yolov9/Citrus_prediction_data-6/train/labels'

# Ensure the path exists before proceeding
if os.path.exists(train_label_folder):
    # Count the number of objects in each class
    class_counts = count_objects(train_label_folder)

    # Output the counts
    for class_label, count in class_counts.items():
        print(f"Total objects in class '{class_label}': {count}")
else:
    print(f"The directory {train_label_folder} does not exist. Please check the path and try again.")


calculate how many images in total we have now

In [ ]:
import os

# Define the paths to the image directories
train_images_path = '/content/yolov9/Citrus_prediction_data-6/train/images'
test_images_path = '/content/yolov9/Citrus_prediction_data-6/test/images'
valid_images_path = '/content/yolov9/Citrus_prediction_data-6/valid/images'

# Function to count the number of image files in a directory
def count_images(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])

# Calculate the number of images in each folder
train_images_count = count_images(train_images_path)
test_images_count = count_images(test_images_path)
valid_images_count = count_images(valid_images_path)

print(f"Number of images in train folder: {train_images_count}")
print(f"Number of images in test folder: {test_images_count}")
print(f"Number of images in valid folder: {valid_images_count}")


In [ ]:
!python /content/yolov9/train.py \
--batch 16 --epochs 10 --img 640 --device 0 --min-items 0 --close-mosaic 15 --patience 5\
--data /content/yolov9/Citrus_3
prediction_data-6/data.yaml \
--weights /content/yolov9/gelan-c.pt \
--cfg /content/yolov9/models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml


In [ ]:
ls /content/yolov9/valid/images


*Visualize training results

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename=f"/content/yolov9/runs/train/exp5/labels.jpg", width=1000)

In [ ]:
Image(filename=f"/content/yolov9/runs/train/exp5/confusion_matrix.png", width=1000)

detect conf 0.1

In [ ]:
!python /content/yolov9/detect.py --img 1280 --conf 0.1 --device 0 --weights /content/yolov9/runs/train/exp/weights/best.pt --source /content/Citrus_prediction_data-6/test/images --save-txt --save-conf


try on images for detection task

In [ ]:
!python /content/yolov9/detect.py --img 1280 --conf 0.1 --device 0 --weights /content/yolov9/runs/train/exp2/weights/best.pt --source /content/gdrive/MyDrive/Thesis/Images/20231216/detection/RGB --save-txt --save-conf


get map claculation on test

In [ ]:
!python /content/yolov9/val.py --img 1280 --conf 0.1 --device 0 --weights /content/yolov9/runs/train/exp2/weights/best.pt --data /content/yolov9/Citrus_prediction_data-5/data.yaml --name test


save the results in drive

In [ ]:
import shutil
import os

# Specify source and destination paths
source_path = '/content/yolov9/Citrus_prediction_data-5'
source_path_runs = '/content/yolov9/runs'
destination_root = '/content/gdrive/MyDrive/Thesis/code/YoloV9/21.5.24'
destination_path = os.path.join(destination_root, '21.5.24')

# Create a new folder within the destination directory
if os.path.exists(destination_path):
    shutil.rmtree(destination_path)
os.makedirs(destination_path)

# Function to copy directory and its contents
def copy_directory(src, dest):
    if not os.path.exists(src):
        print(f"Source path does not exist: {src}")
        return

    try:
        shutil.copytree(src, dest)
        print(f"Copied {src} to {dest} successfully!")
    except Exception as e:
        print(f"Error copying {src} to {dest}: {e}")

# Copy the folders to the new directory
copy_directory(source_path_runs, os.path.join(destination_path, 'runs'))
copy_directory(source_path, os.path.join(destination_path, 'data'))

# Check if the images folder exists and list its contents
test_images_path = os.path.join(source_path, 'test/images')
if os.path.exists(test_images_path):
    print("Test images directory exists. Listing contents:")
    for root, dirs, files in os.walk(test_images_path):
        for file in files:
            print(os.path.join(root, file))
else:
    print("Test images directory does not exist.")

print("Folders copied to Google Drive successfully!")


In [ ]:
# Directory containing the detection result files
results_dir = '/content/yolov9/Citrus_prediction_data-5/test/labels'

# Initialize counters
below_threshold_count = 0
above_threshold_count = 0

# Iterate over each detection result file
for filename in os.listdir(results_dir):
    if filename.endswith('.txt'):
        file_path = os.path.join(results_dir, filename)

        # Read the contents of the file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Count predictions below and above the threshold
        for line in lines:
            _, confidence, *_ = line.strip().split()
            if float(confidence) < 0.4:
                below_threshold_count += 1
            else:
                above_threshold_count += 1

print("Number of predictions below 0.4 confidence:", below_threshold_count)
print("Number of predictions above or equal to 0.4 confidence:", above_threshold_count)


In [ ]:
import os

# Specify the directory path
directory = '/content/yolov9'

# Walk through the directory and its subdirectories
for root, dirs, _ in os.walk(directory):
    # Print the current directory
    print(f"Directory: {root}")

    # Print all subdirectories in the current directory
    for dir in dirs:
        print(f"Subdirectory: {os.path.join(root, dir)}")



# **Evaluate best model**

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import os
import torch
from torchvision import datasets, transforms

best wights 2023

In [ ]:
model_weights_path = "/content/gdrive/Othercomputers/My PC/Thesis/Yolo/YoloV8/2023/Original/runs/obb/train/weights/best.pt"
data_yaml_path = "/content/citrus_prediciton_data_2024-1/data.yaml"
model = YOLO(model_weights_path)

# train 2024 - (best original 2023)
try:
    # Place the validation code here to catch any exceptions and print relevant data
    results = model.val(data='/content/citrus_prediciton_data_2024-1/data.yaml', split='train')
except IndexError as e:
    print("Error occurred:", e)

Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /content/citrus_prediciton_data_2024-1/train/labels.cache... 1168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1168/1168 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/10_e_2_20240927_RGB_JPG.rf.a8e27f810e9dcac7cea3b84806ba3914.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/11_w_2_20240927_RGB_JPG.rf.39de69800bbb4d366617295ac9eb92b5.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/1D3A5162_JPG.rf.50937420e440ad1ebf9e1acfb62c5f6d.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:24<00:00,  2.99it/s]


                   all       1168       1457      0.769      0.688      0.741      0.705
      Cracked_Mandarin         85         86      0.613      0.453      0.525       0.48
              Mandarin       1106       1371      0.925      0.923      0.957      0.929
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/obb/val23


best augment 2023

In [ ]:
model_weights_path = "/content/gdrive/Othercomputers/My PC/Thesis/Yolo/YoloV8/2023/augment/runs_6.3/obb9/train/weights/best.pt"
data_yaml_path = "/content/citrus_prediciton_data_2024-1/data.yaml"
model = YOLO(model_weights_path)

# train 2024 - (best augmented 2023)
try:
    # Place the validation code here to catch any exceptions and print relevant data
    results = model.val(data='/content/citrus_prediciton_data_2024-1/data.yaml', split='train')
except IndexError as e:
    print("Error occurred:", e)

Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)


Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:1

YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /content/citrus_prediciton_data_2024-1/train/labels.cache... 1168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1168/1168 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/10_e_2_20240927_RGB_JPG.rf.a8e27f810e9dcac7cea3b84806ba3914.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/11_w_2_20240927_RGB_JPG.rf.39de69800bbb4d366617295ac9eb92b5.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/citrus_prediciton_data_2024-1/train/images/1D3A5162_JPG.rf.50937420e440ad1ebf9e1acfb62c5f6d.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:24<00:00,  3.00it/s]


                   all       1168       1457      0.876      0.707       0.78      0.735
      Cracked_Mandarin         85         86      0.814        0.5      0.607       0.54
              Mandarin       1106       1371      0.939      0.915      0.954      0.931
Speed: 0.4ms preprocess, 4.1ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to runs/obb/val24


best original 2024

In [14]:
model_weights_path = "/content/gdrive/Othercomputers/My PC/Thesis/Yolo/YoloV8/2024/original/runs/obb_2024/train/weights/best.pt"
data_yaml_path = "/content/datasets/Citrus_prediction_data-8/data.yaml"
model = YOLO(model_weights_path)

# train 2023 - (best augmented 2024)
try:
    # Place the validation code here to catch any exceptions and print relevant data
    results = model.val(data='/content/datasets/Citrus_prediction_data-8/data.yaml', split='train')
except IndexError as e:
    print("Error occurred:", e)

Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /content/datasets/Citrus_prediction_data-8/train/labels... 817 images, 0 backgrounds, 0 corrupt: 100%|██████████| 817/817 [00:00<00:00, 1972.71it/s]

val: New cache created: /content/datasets/Citrus_prediction_data-8/train/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:13<00:00,  3.73it/s]


                   all        817        873      0.837      0.645      0.682      0.675
      Cracked_Mandarin         32         32       0.71      0.344      0.395      0.392
              Mandarin        785        841      0.963      0.946      0.968      0.958
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to runs/obb/val2


best augment 2024

In [11]:
model_weights_path = "/content/runs/obb_2024/train/weights/best.pt"
data_yaml_path = "/content/Citrus_prediction_data-8/data.yaml"
model = YOLO(model_weights_path)

# train 2023 - (best augmented 2024)
try:
    # Place the validation code here to catch any exceptions and print relevant data
    results = model.val(data='/content/Citrus_prediction_data-8/data.yaml', split='train')
except IndexError as e:
    print("Error occurred:", e)

Ultralytics YOLOv8.2.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /content/Citrus_prediction_data-8/train/labels... 817 images, 0 backgrounds, 0 corrupt: 100%|██████████| 817/817 [00:00<00:00, 1238.67it/s]

val: New cache created: /content/Citrus_prediction_data-8/train/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:16<00:00,  3.07it/s]


                   all        817        873      0.789      0.691      0.705      0.699
      Cracked_Mandarin         32         32      0.639      0.406      0.442      0.442
              Mandarin        785        841      0.938      0.976      0.968      0.956
Speed: 0.3ms preprocess, 4.6ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to runs/obb/val
